In [16]:
from pathlib import Path
import simplekml
import numpy as np

In [17]:
coords = Path('data/testing_points.csv')
kml = simplekml.Kml()
with coords.open() as f:
    lls = f.read().split()
    lls = [float(ll[:-1]) for ll in lls]
lls = np.array(lls).reshape(-1, 2)
lls

array([[  50.646936, -145.997698],
       [  54.6677  , -139.010041],
       [  54.477458, -141.616691],
       [  54.383362, -141.045229],
       [  53.814698, -139.358477],
       [  53.569767, -148.276989],
       [  53.282177, -148.044411],
       [  51.349113, -147.937332],
       [  51.668881, -137.179582],
       [  51.270367, -136.826307],
       [  51.030979, -136.234205],
       [  51.048706, -135.453401],
       [  51.601618, -133.363832],
       [  51.338698, -132.712944],
       [  50.525618, -132.628394],
       [  50.413418, -134.881734],
       [  49.916068, -133.764702],
       [  49.641879, -133.418858],
       [  49.338347, -133.293898],
       [  49.183028, -133.500274],
       [  48.973656, -136.979826],
       [  51.024592, -136.236683],
       [  49.736682, -160.346734],
       [  48.93099 , -156.050723],
       [  48.93099 , -156.050723],
       [  51.365232, -148.128561],
       [  46.924423, -159.272355],
       [  47.960986, -145.541106],
       [  47.897147,

In [18]:
for ll in lls:
    kml.newpoint(coords=[(ll[1], ll[0])])
kml.save('out/handpick_test_coords.kml')